## Example: Pricing of European Options Contracts
Let's begin by computing the premium $\mathcal{P}_{c}(K,S(0))$ the buyer must pay for a `call` contract. When early excersise is not allowed, the pricing formula is given by:

$$\mathcal{P}_{c}(K,S(0)) = \mathbb{E}\Bigl(\mathcal{D}^{-1}_{T,0}(\bar{r})\cdot{V_{c}}(K,S(T))\Bigr)$$

which says the right (but not the obligation) to excercise the `call` contract is the expected value of the discounted future payoff from the contract. To compute the expectation, we simulate the future share price `T` days in the future using a [geometric Brownian motion](https://en.wikipedia.org/wiki/Geometric_Brownian_motion#:~:text=A%20geometric%20Brownian%20motion%20(GBM,a%20Wiener%20process)%20with%20drift.) model for the future share price:

$$S(T) = S(0)\cdot\exp\Biggl[\left(\bar{r}-\frac{\sigma^{2}}{2}\right)\cdot{T} + (\sigma\sqrt{T})\cdot{Z_{t}(0,1)}\Biggr]$$

where we assume $S(0)$ is the share price today, $\bar{r}$ denotes the risk-free rate (risk-neutral pricing), $\sigma$ denotes share price volatility and `T` denotes the number of days until contract expiration. Once we have the $\mathcal{P}_{c}(K,S(0))$ value, we can calculate (and visualize) the payoff, potential profit and breakeven values for the `call` contract.

### Learning Objectives
Let's consider an example: We have a European `call` that expires in `T = 365` days with a strike price `K = 60.0`. The current share price is `S(0)=60.0`. Assume a risk free rate of return of `5%` and a implied volatility of `10%`. The correct answer is $\mathcal{P}_{c}(K,S(0))$ = 4.08 USD/share. 

Let's compute the European `call` premium using two appoaches:

* Approach 1: Monte Carlo simulation of the share price by sampling the geometric Brownian motion model, and then explicitly calculating the expectation in the pricing relationship
* Approach 2: Use the [Black–Scholes](https://en.wikipedia.org/wiki/Black–Scholes_model) pricing formula for a European `call` option

## Setup

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5660-Examples-F23`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`


securityterm (generic function with 1 method)

### Constants

Define the problem parameters that are shared between the approaches:

In [2]:
Δt = (1.0/365.0);
Sₒ = 60.0;
K = 60.0;
T = 365.0*Δt;
r̄ = 0.05;
σ̄ = 0.10;

## Approach 1: Monte Carlo estimate of European `call` contract price
Let's build an instance of the `MyGeometricBrownianMotionEquityModel` type which holds the value for the price simulation using the `build(...)` method and store this instance in the `model` variable:

In [3]:
model = build(MyGeometricBrownianMotionEquityModel, (
        μ = r̄, σ = σ̄));

Next, we build an instance of the `MyEuropeanCallContractModel` type which holds the parameters for the `call` contract using the `build(...)` method. We store the contract model in the `call_contract_model` variable:

In [4]:
call_contract_model = build(MyEuropeanCallContractModel, (
        K = K, IV = σ̄, DTE = T, sense = 1));

We'll simulate the future share price at expiration $S(T)$ for different number of sample paths stored in the `number_of_samples` array: